In [2]:
#Guide for AdaptVQE -> Ground State
# for Natural Science Problems
!pip install qiskit-nature
!pip install --prefer-binary pyscf
!pip install sparse



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
#First need ElectronicStructureProblem to solve:
from qiskit_nature.second_q.drivers import PySCFDriver
driver = PySCFDriver(atom='H 0 0 0 ; H 0 0 0.735', basis='sto-3g')
problem = driver.run() # start problem
from qiskit_nature.second_q.mappers import JordanWignerMapper
mapper = JordanWignerMapper()

# problem + mapper -> ansatz


/home/codespace/.python/current/lib/python3.10/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [3]:
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
    )
)

In [4]:
#VQE Setup
import numpy as np
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
vqe = VQE(Estimator(), ansatz, SLSQP())
vqe.initial_point = np.zeros(ansatz.num_parameters) #asically same

#AdaptVQE setup
from qiskit_algorithms import AdaptVQE #converts the Vqe
adapt_vqe = AdaptVQE(vqe)
adapt_vqe.supports_aux_operators = lambda: True #not sure here



In [5]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
solver = GroundStateEigensolver(mapper, adapt_vqe)

#mapper + vqe -> solver


In [6]:
result = solver.solve(problem)

print(f"Total ground state energy = {result.total_energies[0]:.4f}")

Total ground state energy = -1.1373


In [ ]:
#Conclusions
# PySCF -> Problem + run()
#Mapper + Problem -> ansatz
# Estimator + ansatz + Optimizer -> VQE
# Variant(VQE) -> Variant version
#mapper + vqe -> solver
#solver.solve(problem)